In [1]:
import datetime

from util import importstr
from logconf import logging

log = logging.getLogger("nb")


def run(app, *argv):
    argv = list(argv)
    argv.insert(0, "--num-workers=4")  # <1>
    log.info("Running: {}({!r}).main()".format(app, argv))

    app_cls = importstr(*app.rsplit(".", 1))  # <2>
    app_cls(argv).main()

    log.info("Finished: {}.{!r}).main()".format(app, argv))


import os
import shutil


# clean up any old data that might be around.
# We don't call this by default because it's destructive,
# and would waste a lot of time if it ran when nothing
# on the application side had changed.
def cleanCache():
    shutil.rmtree("data-unversioned/cache")
    os.mkdir("data-unversioned/cache")


# cleanCache()

In [2]:
training_epochs = 20
experiment_epochs = 10
final_epochs = 50

training_epochs = 2
experiment_epochs = 2
final_epochs = 5
seg_epochs = 10

In [3]:
run("prepcache.LunaPrepCacheApp")

2024-11-03 10:57:59,088 INFO     pid:19542 nb:012:run Running: prepcache.LunaPrepCacheApp(['--num-workers=4']).main()
2024-11-03 10:58:06,137 INFO     pid:19542 prepcache:051:main Starting LunaPrepCacheApp, Namespace(batch_size=1024, num_workers=4)
2024-11-03 10:58:07,024 INFO     pid:19542 dsets:319:__init__ <dsets.LunaDataset object at 0x7fdc3c1747d0>: 56928 training samples, 56816 neg, 112 pos, unbalanced ratio
2024-11-03 10:58:07,024 WARNING  pid:19542 util:225:enumerateWithEstimate Stuffing cache ----/56, starting
2024-11-03 10:58:26,844 INFO     pid:19542 util:245:enumerateWithEstimate Stuffing cache    8/56, done at 2024-11-03 11:00:19, 0:02:04
2024-11-03 10:58:41,048 INFO     pid:19542 util:245:enumerateWithEstimate Stuffing cache   16/56, done at 2024-11-03 10:59:59, 0:01:44
2024-11-03 10:59:08,365 INFO     pid:19542 util:245:enumerateWithEstimate Stuffing cache   32/56, done at 2024-11-03 10:59:50, 0:01:35
2024-11-03 10:59:43,485 WARNING  pid:19542 util:260:enumerateWithEstim

In [4]:
run("training.ClassificationTrainingApp", f"--epochs=20", "nodule-nonnodule")

2024-11-03 10:59:43,498 INFO     pid:19542 nb:012:run Running: training.ClassificationTrainingApp(['--num-workers=4', '--epochs=20', 'nodule-nonnodule']).main()
2024-11-03 10:59:43,687 INFO     pid:19542 training:157:initModel Using CUDA; 1 devices.
2024-11-03 10:59:43,976 INFO     pid:19542 training:223:main Starting ClassificationTrainingApp, Namespace(batch_size=24, num_workers=4, epochs=20, dataset='LunaDataset', model='LunaModel', malignant=False, finetune='', finetune_depth=1, tb_prefix='p2ch14', comment='nodule-nonnodule')
2024-11-03 10:59:43,991 INFO     pid:19542 dsets:319:__init__ <dsets.LunaDataset object at 0x7fdc3c1a2310>: 50616 training samples, 50515 neg, 101 pos, 1:1 ratio
2024-11-03 10:59:43,996 INFO     pid:19542 dsets:319:__init__ <dsets.LunaDataset object at 0x7fdbda14ba90>: 6312 validation samples, 6301 neg, 11 pos, unbalanced ratio
2024-11-03 10:59:43,997 INFO     pid:19542 training:232:main Epoch 1 of 20, 2084/263 batches of size 24*1
2024-11-03 10:59:44,014 WARN

In [5]:
import numpy as np

np.int = np.int32
np.bool = np.bool_
run("nodule_analysis.NoduleAnalysisApp", "--run-validation")

2024-11-03 18:55:48,923 INFO     pid:30862 nb:012:run Running: nodule_analysis.NoduleAnalysisApp(['--num-workers=4', '--run-validation']).main()
2024-11-03 18:55:48,924 DEBUG    pid:30862 nodule_analysis:254:initModels /home/lim/Desktop/other/ML/pytorch_dlwpt-code-master/data/part2/models/seg_2020-01-26_19.45.12_w4d3c1-bal_1_nodupe-label_pos-d1_fn8-adam.best.state
/home/lim/Desktop/other/ML/lesson14/nodule_analysis.py:255: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unl

1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |              982 |               18
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                0
1.3.6.1.4.1.14519.5.2.1.6279.6001.130438550890816550994739120843
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             3669 |               10
          Benign |                0 |                0 |                1
       Malignant |                0 |                0 |                0
1.3.6.1.4.1.14519.5.2.1.6279.6001.187451715205085403623595258748
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |              565 |                5
          Benign |                0 |                0 |         

2024-11-03 18:56:29,186 INFO     pid:30862 util:245:enumerateWithEstimate Series    4/9, done at 2024-11-03 18:57:01, 0:01:12


1.3.6.1.4.1.14519.5.2.1.6279.6001.277445975068759205899107114231
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |              511 |                7
          Benign |                0 |                0 |                2
       Malignant |                0 |                0 |                0
1.3.6.1.4.1.14519.5.2.1.6279.6001.305858704835252413616501469037
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             2102 |               22
          Benign |                1 |                0 |                1
       Malignant |                0 |                0 |                0
1.3.6.1.4.1.14519.5.2.1.6279.6001.332453873575389860371315979768
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1011 |               15
          Benign |                0 |                0 |         

2024-11-03 18:57:11,603 INFO     pid:30862 util:245:enumerateWithEstimate Series    8/9, done at 2024-11-03 18:57:11, 0:01:22
2024-11-03 18:57:11,603 WARNING  pid:30862 util:260:enumerateWithEstimate Series ----/9, done at 2024-11-03 18:57:11
2024-11-03 18:57:11,606 INFO     pid:30862 nb:017:run Finished: nodule_analysis.NoduleAnalysisApp.['--num-workers=4', '--run-validation']).main()


1.3.6.1.4.1.14519.5.2.1.6279.6001.752756872840730509471096155114
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |              808 |                8
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                1
Total
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |            13773 |              125
          Benign |                1 |                0 |                5
       Malignant |                0 |                0 |                5
